In [2]:
from sklearn.model_selection import train_test_split

from utils.prepare_data import read_data, save_data, target_columns

In [3]:
df = read_data(snakemake.input["sim_params"])

In [5]:
train, valid = train_test_split(
    df,
    test_size=snakemake.config["validation_percentage"],
    random_state=snakemake.params["random_seed"],
    shuffle=True,
    stratify=df.regime
)

In [12]:
save_data(train, snakemake.output["training"])
save_data(valid, snakemake.output["validation"])